<a href="https://colab.research.google.com/github/IamArmanNikkhah/Producer-Consumer/blob/main/producer_consumer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from threading import Thread, Semaphore
import time
import random
import string
import math

In [ ]:
queue = []         #queue from where producer will produce data and consumer will consume data
MAX_NUM = 10       #max limit of the queue
mutex = Semaphore(1)
empty = Semaphore(MAX_NUM)
full  = Semaphore(0)
flag  = True

In [ ]:
  class SenderWorkerThread(Thread):

    def __init__(self):
      super(SenderWorkerThread, self).__init__()
      self.workerID = "senderThread-" + str(math.floor(random.random()*100)) + "#"
     
    def run(self):
      global queue
      global flag    
      while flag:
            item = self.random_string_produce()
            empty.acquire()
            mutex.acquire()  #wait operation to stop consuming  
            queue.append(item) 
            print(str(time.process_time()),self.workerID, " Finished Writing") 
            mutex.release() #signal operation to allow consumer to consume data
            full.release()
            time.sleep(random.random()) #to allow program to run a bit slower 

    def random_string_produce(self):
      chars=string.ascii_letters + string.digits   
      return ''.join(random.choice(chars) for _ in range(random.randint(1,10)))

In [ ]:
class SenderMainThread(Thread):
  def __init__(self, s = 10, t = 1):
    super(SenderMainThread, self).__init__()
    self.slots_number = s
    self.theards_number = t
    self.Workertheards = [SenderWorkerThread() for _ in range(self.theards_number)]
  
  def run(self):
    for i in range(self.theards_number):
      self.Workertheards[i].start()         
    for j in range(self.theards_number):    # need some work
      self.Workertheards[j].join()

In [ ]:
queue_receivers = []
mutex_receivers = Semaphore(1)

In [ ]:
class ReceiverWorkerThread(Thread):
  
  def __init__(self):
    super(ReceiverWorkerThread, self).__init__()
    self.workerID = "receiverThread-" + str(math.floor(random.random()*100)) + "#"
  
  def run(self):
    global queue
    global flag 
    while flag:
      full.acquire()
      mutex.acquire()
      print(str(time.process_time()),self.workerID, " started reading") 
      item = queue.pop(0)
      mutex.release() 
      empty.release()
      self.consume_item(item) 
      time.sleep(random.random()) #to allow program to run a bit slower
    
  
  def consume_item(self, item):
    global queue_receivers
    mutex_receivers.acquire()
    queue_receivers.append(item)
    mutex_receivers.release()

In [ ]:
class ReceiverMainThread(Thread):
  def __init__(self,s = 10,t = 1,N:list = []):
    super(ReceiverMainThread, self).__init__()
    self.slots_number = s
    self.theards_number = t
    self.N = N
    self.Workertheards = [ReceiverWorkerThread() for _ in range(self.theards_number)]

  def run(self):
    
    for i in range(self.theards_number):
      self.Workertheards[i].start()            
    t1 = Thread(target=self.checkMEM)
    t1.start()
    t1.join()
    for j in range(self.theards_number):    
      self.Workertheards[j].join()


  def checkMEM(self):
    N_copy = self.N
    global queue_receivers 
    global flag 
    while(N_copy):
      num = N_copy[0]
      mutex_receivers.acquire()
      #print(queue_receivers)              ##TEST
      if len(queue_receivers) >= num:      #also check ==
        N_copy.pop(0)
        out_str = ""
        for _ in range(num):
          out_str = out_str + queue_receivers.pop(0)
        print("% GENERATED TEXT IS : ",out_str)
      mutex_receivers.release()
      time.sleep(random.random())
    flag = False
    print("# Program finished successfully! # ") 
    

In [ ]:
if __name__ == '__main__':
  receiver = ReceiverMainThread(10,3,[4,2,3,3])
  sender   = SenderMainThread(10,2)
  sender.start()
  receiver.start()
  sender.join()
  receiver.join()